In [22]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [23]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [24]:
""" We need different stocks market indicators in comparison to the equal weight strat
    We need price and 1 year stock return in place of market cap"""

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()

"""To parse a .json object like this you pass ['key']"""

data['year1ChangePercent']

0.2690710790734838

In [25]:
def chunks(lst, n):
    """chunks of size en off the list"""
    for i in range(0, len(lst), n):
        yield lst[i: i+n]
        
my_columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))



In [26]:
final_dataframe = pd.DataFrame(columns=my_columns)


for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    " This loop takes care of appending the relevant metrics to the final dataframe "
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index = True
        )

final_dataframe
    

,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,A,158.760,0.539709,N/A
1,AAL,20.710,0.664011,N/A
2,AAP,220.100,0.394979,N/A
3,AAPL,147.195,0.273016,N/A
4,ABBV,112.240,0.338399,N/A
...,...,...,...,...
500,YUM,129.020,0.371168,N/A
501,ZBH,150.290,0.044451,N/A
502,ZBRA,522.120,0.90887,N/A
503,ZION,63.200,1.137146,N/A


In [27]:
""" Now lets get rid of the low momentum stocks 
    First we have to rearrange the stocks in descending order
    looking at the One-Year Price Return column
    The inplace method sets the modification on the dataframe"""

final_dataframe.sort_values('One-Year Price Return', ascending= False, inplace= True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,39.616,3.162453,N/A
1,317,MRO,15.955,2.905458,N/A
2,175,FANG,106.800,2.700094,N/A
3,275,LB,83.590,2.330877,N/A
4,357,OXY,33.200,2.267966,N/A
5,299,MCHP,152.648,1.810655,N/A
6,111,COTY,8.360,1.760238,N/A
7,410,SIVB,683.560,1.624353,N/A
8,195,FTNT,320.900,1.618847,N/A
9,160,EOG,87.100,1.558159,N/A


In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio :')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \n Please try again :')
        portfolio_size = input('Enter the size of your portfolio :')
    
portfolio_input()
print(portfolio_)